### References
- Multi-Label Classification Model From Scratch: Step-by-Step Tutorial (https://huggingface.co/blog/Valerii-Knowledgator/multi-label-classification)
- https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb
- https://github.com/huggingface/notebooks/blob/main/examples/text_classification.ipynb

# Libraries

In [1]:
%pip install -q -U datasets transformers accelerate sentencepiece

In [2]:
import os
import random
import numpy as np
import torch
import transformers
from pprint import pprint
from datetime import datetime
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments
from sklearn.metrics import f1_score

# Config

In [3]:
seed = 42
lang = 'sun'
project_name = f'RoBERTa-Base-SE2025T11A-{lang}-v{datetime.now().strftime("%Y%m%d%H%M%S")}'
print("Project name:", project_name)

# hf_model_id = 'bhadresh-savani/roberta-base-emotion'
# hf_model_id = 'bhadresh-savani/bert-base-uncased-emotion'
# hf_model_id = 'w11wo/sundanese-bert-base-emotion-classifier'
hf_model_id = 'w11wo/sundanese-roberta-base-emotion-classifier'
# hf_model_id = 'w11wo/sundanese-roberta-base'
# hf_model_id = 'alxxtexxr/XLM-RoBERTa-Base-Sundanese-Emotion-Classifier-v20241222170134'
hf_tokenizer_id = hf_model_id
hf_data_id = 'alxxtexxr/SemEval2025-Task11-Dataset'
# hf_data_config = 'track_a_sun_70_15_15_stratify_v2'
hf_data_config = 'track_a_sun_go_emotions_70_15_15_stratify_v2'
# hf_data_config = 'track_a_sun_go_emotions_70_15_15_balanced'

num_epochs = 10

Project name: RoBERTa-Base-SE2025T11A-sun-v20250110165021


In [4]:
def set_seed(seed):
    # Set random seed for NumPy
    np.random.seed(seed)

    # Set random seed for Torch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensures deterministic results
    torch.backends.cudnn.benchmark = False  # Avoids non-deterministic algorithms

    # Set random seed for Transformers
    transformers.set_seed(seed)

    # Optionally set random seed for sklearn and Python's own random module
    random.seed(seed)

    # Set random seed for os
    os.environ['PYTHONHASHSEED'] = str(seed)

    print(f"Random seed set to: {seed}")

set_seed(seed)

Random seed set to: 42


# Data

## Load Data

In [5]:
datasets = load_dataset(hf_data_id, hf_data_config)

cols = list(datasets['train'].features)
emotion_cols = [col for col in cols if col not in ['Unnamed: 0', 'text', 'emotion']]
splits = [*datasets.keys()]

print("Data columns:", cols)
print("Emotions columns:", emotion_cols)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Data columns: ['text', 'emotion', 'marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']
Emotions columns: ['marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']


In [6]:
class2id = {class_:id for id, class_ in enumerate(emotion_cols)}
id2class = {id:class_ for class_, id in class2id.items()}

print("Class to ID:")
pprint(class2id, width=1)
print()
print("ID to Class:")
pprint(id2class, width=1)

Class to ID:
{'biasa': 6,
 'jijik': 1,
 'marah': 0,
 'sedih': 4,
 'senang': 3,
 'takut': 2,
 'terkejut': 5}

ID to Class:
{0: 'marah',
 1: 'jijik',
 2: 'takut',
 3: 'senang',
 4: 'sedih',
 5: 'terkejut',
 6: 'biasa'}


## Preprocess Data

In [7]:
tokenizer = AutoTokenizer.from_pretrained(hf_tokenizer_id)

In [8]:
def one_hot_encode_emotion(emotion, emotion_cols):
    emotions = emotion.replace(" ", "").split(",")
    one_hot_emotion = [1.0 if emotion_col in emotions else 0.0 for emotion_col in emotion_cols] # Ensure that the label is float, not int
    return one_hot_emotion

def preprocess_function(data):
   text = data['text']
   emotion = data['emotion']
   labels = one_hot_encode_emotion(emotion, emotion_cols)
   data = tokenizer(text, truncation=True)
   data['labels'] = labels
   return data

tokenized_datasets = {split: datasets[split].map(preprocess_function) for split in splits}

In [9]:
# Sanity check
data = tokenized_datasets['train'][5]

print("Text:", data['text'])
print("Emotion(s):", data['emotion'])
print("Labels:", data['labels'], '-->', emotion_cols)

Text: Sigana anjeun lalaki séjén anu henteu resep ka legion ;D
Emotion(s): marah, jijik
Labels: [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0] --> ['marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']


In [10]:
data_collator = DataCollatorWithPadding(tokenizer)

# Model

In [11]:
# model = AutoModelForSequenceClassification.from_pretrained(
#     hf_model_id, 
#     num_labels=len(emotion_cols),
#     id2label=id2class, 
#     label2id=class2id,
#     problem_type = "multi_label_classification",
# )

################ TRANSFER LEARNING ################
model = AutoModelForSequenceClassification.from_pretrained(
    hf_model_id, 
    # num_labels=len(emotion_cols),
    # id2label=id2class, 
    # label2id=class2id,
    problem_type = "multi_label_classification",
)
print(model.classifier)

model.classifier.out_proj = torch.nn.Linear(in_features=768, out_features=len(emotion_cols), bias=True)
print(model.classifier)

# Freeze the rest of the layers for transfer learning
# for param in model.parameters():
#     param.requires_grad = False

RobertaClassificationHead(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (out_proj): Linear(in_features=768, out_features=4, bias=True)
)
RobertaClassificationHead(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (out_proj): Linear(in_features=768, out_features=7, bias=True)
)


# Finetuning

In [12]:
def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    probs = sigmoid(predictions)
    y_pred = (probs > 0.5).astype(int)
    y_true = labels.astype(int)

    # Compute F1 score for each type of averaging method
    f1_micro = f1_score(y_true, y_pred, average='micro', zero_division=0.0)
    f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0.0)
    f1_weighted = f1_score(y_true, y_pred, average='weighted', zero_division=0.0)
    f1_samples = f1_score(y_true, y_pred, average='samples', zero_division=0.0)
    f1_labels = f1_score(y_true, y_pred, average=None, zero_division=0.0)
    f1_labels_dict = {f'f1_label_{emotion_cols[i]}': f1_labels[i] for i in range(len(f1_labels))}

    return {
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
        'f1_weighted': f1_weighted,
        'f1_samples': f1_samples,
        **f1_labels_dict,
    }

In [13]:
train_args = TrainingArguments(
    # Training config
    per_device_train_batch_size=2,
    num_train_epochs=num_epochs,
    learning_rate=2e-5,
    weight_decay=0.01,

    # Logging config for training
    logging_strategy='steps',
    logging_steps=100,

    # Evaluation config during training
    per_device_eval_batch_size=2,
    eval_strategy='steps',
    eval_steps=100,

    # Model saving config
    output_dir=project_name,
    save_strategy='epoch',
    # load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-13-dc5948f208f6>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: alimtegar. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,F1 Macro,F1 Micro,F1 Weighted,F1 Samples,F1 Label Marah,F1 Label Jijik,F1 Label Takut,F1 Label Senang,F1 Label Sedih,F1 Label Terkejut,F1 Label Biasa
100,0.491100,0.418400,0.207094,0.365105,0.261328,0.273242,0.000000,0.000000,0.200000,0.741463,0.508197,0.000000,0.000000
200,0.426200,0.382818,0.204096,0.384743,0.265081,0.284932,0.119403,0.000000,0.036364,0.794643,0.478261,0.000000,0.000000
300,0.378300,0.364250,0.292809,0.419301,0.349729,0.315982,0.345679,0.000000,0.312500,0.781726,0.341463,0.268293,0.000000
400,0.391600,0.340722,0.352057,0.446254,0.404034,0.337443,0.300000,0.092308,0.376812,0.703030,0.666667,0.325581,0.000000
500,0.368200,0.315451,0.436597,0.552846,0.491616,0.505023,0.513514,0.035088,0.528736,0.804020,0.745763,0.377778,0.051282
600,0.343700,0.322685,0.487361,0.568681,0.535550,0.504566,0.387755,0.504505,0.567901,0.771739,0.754098,0.425532,0.000000
700,0.316700,0.302715,0.578916,0.616993,0.598931,0.587671,0.557692,0.489796,0.580645,0.800000,0.689655,0.425532,0.509091
800,0.339100,0.289645,0.574444,0.634465,0.609694,0.598356,0.564516,0.384615,0.575000,0.851485,0.770492,0.500000,0.375000
900,0.355600,0.296888,0.565818,0.614396,0.593310,0.574429,0.465116,0.514286,0.545455,0.834225,0.736111,0.425532,0.440000
1000,0.262100,0.288102,0.636676,0.667513,0.654149,0.647215,0.605042,0.574257,0.581395,0.840580,0.730435,0.514851,0.610169


TrainOutput(global_step=8830, training_loss=0.11773580600964254, metrics={'train_runtime': 1077.947, 'train_samples_per_second': 16.374, 'train_steps_per_second': 8.191, 'total_flos': 201911854570200.0, 'train_loss': 0.11773580600964254, 'epoch': 10.0})

In [15]:
trainer.push_to_hub()

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

events.out.tfevents.1736527839.b5bf64fdeef9.31020.0:   0%|          | 0.00/103k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20250110165021/commit/897e88f743d9a900923767501dba239d8b6037bd', commit_message='End of training', commit_description='', oid='897e88f743d9a900923767501dba239d8b6037bd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20250110165021', endpoint='https://huggingface.co', repo_type='model', repo_id='alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20250110165021'), pr_revision=None, pr_num=None)

# Evaluation

In [18]:
eval = trainer.evaluate(eval_dataset=tokenized_datasets['test'])

print("Evaluation to copy:")
f1_keys = [eval_key for eval_key in eval.keys() if 'f1' in eval_key]
for i, k in enumerate(f1_keys): print(eval[k], end="\t" if i + 1 < len(f1_keys) else "")
print()
print()
print("Evaluation full results:")
pprint(eval)

Evaluation to copy:
0.6371177852359426	0.6569506726457399	0.653192358205208	0.6778995433789955	0.5625	0.4909090909090909	0.7254901960784313	0.8374384236453202	0.7205882352941176	0.5362318840579711	0.5866666666666667

Evaluation full results:
{'epoch': 10.0,
 'eval_f1_label_biasa': 0.5866666666666667,
 'eval_f1_label_jijik': 0.4909090909090909,
 'eval_f1_label_marah': 0.5625,
 'eval_f1_label_sedih': 0.7205882352941176,
 'eval_f1_label_senang': 0.8374384236453202,
 'eval_f1_label_takut': 0.7254901960784313,
 'eval_f1_label_terkejut': 0.5362318840579711,
 'eval_f1_macro': 0.6371177852359426,
 'eval_f1_micro': 0.6569506726457399,
 'eval_f1_samples': 0.6778995433789955,
 'eval_f1_weighted': 0.653192358205208,
 'eval_loss': 0.5267201662063599,
 'eval_runtime': 2.5149,
 'eval_samples_per_second': 145.132,
 'eval_steps_per_second': 72.765}


# Inference

In [17]:
data = datasets['val'][1]
text = data['text']
emotion_true = data['emotion']

inputs = tokenizer(text, return_tensors='pt').to(model.device)

outputs = trainer.model(**inputs)
logits = outputs.logits
probs = sigmoid(logits.squeeze().detach().cpu().numpy()) # apply sigmoid + threshold
labels_pred = (probs > 0.5).astype(int)
emotion_pred = [id2class[idx] for idx, label in enumerate(labels_pred) if label == 1.0] # turn predicted id's into actual label names

print("Text:", text)
print("True emotion(s):", emotion_true)
print("Predicted emotion(s):", ", ".join(emotion_pred))

Text: Abdi hanjakal pisan
True emotion(s): sedih
Predicted emotion(s): sedih
